# NEXUS tool: case study for the Souss-Massa basin - energy demand calculations
In this notebook a case study for the Souss-Massa basin is covered using the `nexus_tool` package. The water requirements for agricultural irrigation, residential, industrial and tourism use were previously calculated using the Water Evaluation and Planning System (WEAP) model. In this case study, the energy requirements for groundwater pumping, wastewater treatment, desalination of seawater and pumping for water conveyance are estimated.

First import the package by running the following block:

In [3]:
%load_ext autoreload

In [4]:
%autoreload
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
import nexus_tool
from nexus_tool.weap_tools import create_folder
from nexus_tool.weap_tools import create_learning_curve
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
# register_matplotlib_converters()
from functools import reduce

## Reading the input files



In [5]:
 #Define the path to read the cropland and builtup are data and reads it in
folder_path = os.path.join('Data')
cropland_path = os.path.join(folder_path, 'Cropland and Builtarea', 'cropland.gz')
cropland = nexus_tool.read_csv(cropland_path)
weap = pd.read_csv(os.path.join('../Morocco dashboard','data', 'Desalination','Climate Change', 'results.gz'))
weap.loc[weap['Province']=='Chtouka-Ait Baha', 'Province'] = 'Chtouka-Aït Baha'
weap.loc[weap['Province']=='Inezgane-Ait Melloul', 'Province'] = 'Inezgane-Aït Melloul'
weap.loc[weap['Province']=='Agadir-Ida-Ou-Tanane', 'Province'] = 'Agadir-Ida ou Tanane'

#weap: the results of the enegy model based on weap demand nodes

cropland = cropland.loc[cropland.Date.isin(weap.Date.unique())] #this should be the results dataframe


In [21]:
summary_provinces_agri = weap.loc[weap['type'].str.contains('Agriculture')].groupby(['Province', 'Date'])[['sswd', 'pwd', 'swpa_e', 'swpp_e']].sum()
temp_cropland_provinces = cropland[['province', 'Date']].copy()
temp_cropland_provinces.loc[temp_cropland_provinces['province']=='Inezgane-Aït Melloul', 'province'] = 'Taroudannt'
for feature in list(summary_provinces_agri):
    cropland[feature] = temp_cropland_provinces.set_index(['province', 'Date']).index.map(summary_provinces_agri[feature]) * cropland['area_share']
        

In [22]:
cropland.rename(columns={'sswd':'water_demand(m3)',
                'pwd':'peak_water(m3)',
                'swpp_e':'peak_load(KW)',
                'swpa_e':'energy_demand(KWh)'}, inplace=True)

In [23]:
cropland.columns

Index(['Year', 'Demand point', 'province', 'area_m2', 'wtd', 'Month', 'srad',
       'wind', 'Date', 'area_share', 'water_demand(m3)', 'peak_water(m3)',
       'energy_demand(KWh)', 'peak_load(KW)'],
      dtype='object')

In [24]:
weap['Province'].unique()

array(['Agadir-Ida ou Tanane', 'Chtouka-Aït Baha', 'Taroudannt', 'Tiznit',
       'Inezgane-Aït Melloul'], dtype=object)

In [25]:
cropland['province'].unique()

array(['Agadir-Ida ou Tanane', 'Chtouka-Aït Baha', 'Inezgane-Aït Melloul',
       'Taroudannt', 'Tiznit'], dtype=object)

## Running Scenarios: 
### This analysis will consider three phase out scenarios for Butane as follows: 
1. Business as usual scebaruio (bau): which assumes the current share of butane use to continue into the future
2. Late Phase out (late_po): which assumes compelte phase out of butane by 2040.
3. Early Phase out (early_po): which assumes compelte phase out of butane by 2030.

For all the three main scenarios, we will explore two levels of PV share in the region.

a. Low PV Share (low_PV): which assumes the current level of PV share of 10% to continue in the future
b. High PV Share (high_PV): whcih assumes an increase of PV share from 10% to 20% by 2040. 

In [70]:
%%time
butane_phaseout_scenarios = [None, 2040, 2030]
pv_levels = [10, 20, 50]
pv_year = 2040

# scenarios = ['late_po']
# pv_levels = ['high_PV']

             
# load_folder = os.path.join('Data','Butane_calculations')
load_folder = os.path.join('test dash results','Butane_calculations')
#results_folder = os.path.join('../Results', 'Butane_results')
#os.makedirs(results_folder, exist_ok = True)
for butane_phaseout in butane_phaseout_scenarios:
    scenario_folder = os.path.join(load_folder, f'phaseout_{butane_phaseout}')
    os.makedirs(scenario_folder, exist_ok = True)
    for pv_level in pv_levels:
        pv_folder = os.path.join(scenario_folder, f'{pv_level}_PV')
        os.makedirs(pv_folder, exist_ok = True)
        # Step1: Reading the input data file
#         file_path = os.path.join(load_folder, 'cropland.csv')
#         df = pd.read_csv(file_path)
        df = cropland
        
        # Step2: Modeling Butane Phase out rate
        def create_tech_dist(iyear, eyear, share1, rates, method, order=2):
            tech_dist = pd.DataFrame({'Year': range(iyear, eyear+1)})
            if method!='step':
                for year, rate in rates.items():
                    tech_dist.loc[tech_dist.Year==year, 'rate'] = rate
                tech_dist['share'] = (1-tech_dist['rate']) * share1
                tech_dist['share'] = tech_dist.share.interpolate(method=method, order=order)
            else:
                for year, rate in rates.items():
                    tech_dist.loc[tech_dist.Year>=year, 'rate'] = rate
                tech_dist['share'] = (1-tech_dist['rate']) * share1
            return tech_dist.set_index('Year').share
        
                        
#         butane_change ={'bau':0,'late_po':1,'early_po':1 }
        if butane_phaseout:
            butane_share = 1
            year = butane_phaseout
        else:
            butane_share = 0
            year = 2050
                
        
# #         end_year ={'bau':2050,'late_po':2040,'early_po':2030 }
#         year = end_year[scenario]
        
#         pv_year ={'bau':2040,'late_po':2040,'early_po':2040}
        
#         pv_change ={'low_PV':0, 'mid_PV':-1, 'high_PV':-4} #The negative sign means increase in share. 
#         pv_share = pv_change[pv_level]
        pv_share = 1 - pv_level / 10
                
        butane_share1 = 0.2
        pv_share1 = 0.1
        df.start_year = 2020
        df.end_year = year
#         pv_year = pv_year[scenario]
        df.mid_year = ((df.start_year+df.end_year)/2)
        bt_rates = {df.start_year: 0, df.mid_year: butane_share*0.5, df.end_year: butane_share}
        pv_rates = {df.start_year: 0, 2030: pv_share*0.5, pv_year: pv_share, 2050: pv_share*1.2}
        butane_dist = create_tech_dist(df.start_year, df.end_year, butane_share1, bt_rates, 'polynomial')
        pv_dist = create_tech_dist(df.start_year, pv_year, pv_share1, pv_rates, 'polynomial')

        #Step3: PV learning curve and crop in PV CAPEX
        def create_learning_curve(iyear, eyear, cc, rates, method, order=2):
            learning_curve = pd.DataFrame({'Year': range(iyear, eyear+1)})
            if method!='step':
                for year, rate in rates.items():
                    learning_curve.loc[learning_curve.Year==year, 'rate'] = rate
                learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
                learning_curve['capital_cost'] = learning_curve.capital_cost.interpolate(method=method, order=order)
            else:
                for year, rate in rates.items():
                    learning_curve.loc[learning_curve.Year>=year, 'rate'] = rate
                learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
            return learning_curve.set_index('Year').capital_cost
        
        pv_rate = 0.4 #Assuming 40% drop in PV capex by 2040.
        cc = 7 #
        #pv_cf = 0.25
        pv_life=15
        df.start_year=2020
        df.end_year= 2050
        #capital_cost = 1000
        rates = {2020: 0, 2030: pv_rate*0.3, 2040: pv_rate, 2050: pv_rate*1.3}
        cc_solar_curve = create_learning_curve(df.start_year, df.end_year, cc, rates, 'polynomial')
        om_cost=0.01 
        efficiency=1
        
        dist = pd.DataFrame({'butane_share':pd.Series(butane_dist),
                   'pv_share':pd.Series(pv_dist),
                   'pv_cc':pd.Series(cc_solar_curve)})
        
        dist['butane_share'].fillna(method='ffill',inplace=True) #This method fills any missing data with the previous value.
        dist['pv_share'].fillna(method='ffill',inplace=True)
        dist['pv_cc'].fillna(method='ffill',inplace=True)
        dist['grid_share']= 1-(dist['pv_share']+dist['butane_share'])
        
        
        souss_massa = pd.merge(df, dist, on='Year')
        
        butane_req = 1/12.58 # LHV = 12.58 KWh/Kg (amount of butane (kg) required to produce 1 KWh) 
        butane_em = 6.67 #kg CO"/Gallon
        gallon2liter = 3.78541 #l/Gallon
        butane_density = 573 #kg/m3
        butane_ef = butane_em / (gallon2liter/1000 * butane_density) #kgCO2/kg
        butane_em_fac = butane_ef * 1000 ##kgCO2/ton
#         butane_em_fac = 3079.5
        bpump_eff = 0.2 #efficiency of butane 
        epump_eff = 0.45 #assumption
        conv_fac = 1000000000 # to convert emissions from kgCO2 to Million meteric tons of CO2 MtCO2

        souss_massa['pv_load(KW)'] = souss_massa['peak_load(KW)']*souss_massa['pv_share']
        souss_massa['pv_demand(KWh)'] = souss_massa['energy_demand(KWh)']*souss_massa['pv_share']
        souss_massa['butane_demand(KWh)'] = souss_massa['energy_demand(KWh)']*souss_massa['butane_share']
        souss_massa['grid_demand(KWh)'] = souss_massa['energy_demand(KWh)']*souss_massa['grid_share']


        souss_massa['pv_elec(KWh)'] = souss_massa['pv_demand(KWh)']/epump_eff
        souss_massa['grid_elec(KWh)'] = souss_massa['grid_demand(KWh)']/epump_eff

        souss_massa['butane_cons(kg)'] = (souss_massa['butane_demand(KWh)']*butane_req)/bpump_eff
        souss_massa['butane_cons(tonnes)'] = souss_massa['butane_cons(kg)']/1000
        souss_massa['butane_emissions(MtCO2)'] = souss_massa['butane_cons(tonnes)'] * butane_em_fac/conv_fac

        souss_massa['butane_FARcost(mMAD)'] = (souss_massa['butane_cons(kg)']*(40/12))/1000000 #in million MAD, this is what farmers pay
        souss_massa['butane_ACTcost(mMAD)'] = (souss_massa['butane_cons(kg)']*(120/12))/1000000 #in million MAD, this is what farmers pay
        souss_massa['butane_SUBSIDY(mMAD)'] = (souss_massa['butane_cons(kg)']*(80/12))/1000000 #in million MAD, this is the total subsidy cost

        
        # Estimating the electriicty from the grid and emissions
        grid_em_fac = 0.7 #kgco2/kwh  , This is based on data from IEA, Morocco 2019 
        grid_cost = 0.57 # MAD/KWh, Assumption and to be updated 

        #scenario1['grid_elec (kwh)'] = scenario1['grid_demand_kwh']/epump_eff #already calculated above
        souss_massa['grid_emissions(MtCO2)'] = souss_massa['grid_elec(KWh)']*grid_em_fac/conv_fac
        souss_massa['grid_cost(mMAD)'] = (souss_massa['grid_elec(KWh)']*grid_cost)/1000000
        souss_massa['Total_emissions(MtCO2)'] = (souss_massa['grid_emissions(MtCO2)'] + souss_massa['butane_emissions(MtCO2)'])

        # Estimating the required monthly capacity of pv NOTE: 1 kWh = 3600 kJ

        souss_massa['cf'] = souss_massa['srad'] / (24*60*60) #This will give the cf in solar rad: (kJ/(m2.day))*30.day/month*1h/(24h*60m*60s) =kWh/(m2.month)*30/(60*60)
        souss_massa['cap_m(MW)'] = souss_massa['pv_load(KW)'] / souss_massa['cf']/1000   #to convert to MW, check the units
        
        souss_massa1 = souss_massa.groupby(['Demand point','Year', 'province', 'area_m2'])[['water_demand(m3)','energy_demand(KWh)', 
                                                                                 'pv_elec(KWh)', 'grid_elec(KWh)',
                                                                                 'butane_cons(tonnes)', 'butane_FARcost(mMAD)', 
                                                                                 'butane_ACTcost(mMAD)','butane_SUBSIDY(mMAD)',
                                                                                 'butane_emissions(MtCO2)','grid_emissions(MtCO2)',
                                                                                 'Total_emissions(MtCO2)','grid_cost(mMAD)',
                                                                                 'pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum()

        #souss_massa1['Province'] = (souss_massa.groupby(['Demand point','Year'])['province'])
        souss_massa1['GWdepth'] = (souss_massa.groupby(['Demand point','Year'])['wtd'].mean())
        souss_massa1['srad'] = (souss_massa.groupby(['Demand point','Year'])['srad'].mean())
        souss_massa1['wind'] = (souss_massa.groupby(['Demand point','Year'])['wind'].mean())
        souss_massa1['cap_m(MW)'] = (souss_massa.groupby(['Demand point','Year'])['cap_m(MW)'].mean())
        souss_massa1['cap_a(MW)'] = (souss_massa.groupby(['Demand point','Year'])['cap_m(MW)'].max())
        souss_massa1['pv_cc'] = (souss_massa.groupby(['Demand point','Year'])['pv_cc'].mean())

        souss_massa1.reset_index(inplace=True)
        souss_massa1.loc[souss_massa1['Year']==2020, 'PV_new_cap(MW)'] = souss_massa1['cap_a(MW)']
        souss_massa1.loc[souss_massa1['Year']!=2020, 'PV_new_cap(MW)'] = souss_massa1['cap_a(MW)'] - souss_massa1['cap_a(MW)'].shift(1)
        souss_massa1.loc[souss_massa1['PV_new_cap(MW)']<0, 'PV_new_cap(MW)'] =0
        
        #PV reinvestment calculations
#         souss_massa2 = pd.DataFrame()
#         for index, group in souss_massa1.groupby(['Demand point']):
#             dff = group.copy()
#             dff['reinv_cap(MW)'] = dff['PV_new_cap(MW)'].shift(pv_life).fillna(0)
#             souss_massa2 = souss_massa2.append(dff, ignore_index=True)
            
#             souss_massa2['PV_Capex(mMAD)']=(souss_massa2['PV_new_cap(MW)']+souss_massa2['reinv_cap(MW)'])*souss_massa2['pv_cc']
#             souss_massa2['PV_Opex(mMAD)']=(souss_massa2['PV_Capex(mMAD)']*om_cost)
        
        souss_massa1['reinv_cap(MW)'] = souss_massa1['PV_new_cap(MW)'].shift(pv_life).fillna(0)
        souss_massa1.loc[souss_massa1['Year']<(2020+pv_life), 'reinv_cap(MW)'] = 0
        souss_massa1['PV_Capex(mMAD)']=(souss_massa1['PV_new_cap(MW)']+souss_massa1['reinv_cap(MW)'])*souss_massa1['pv_cc']
        souss_massa1['PV_Opex(mMAD)']=(souss_massa1['PV_Capex(mMAD)']*om_cost)

        souss_massa1_summary = souss_massa1.groupby(['Year'])[['water_demand(m3)','energy_demand(KWh)', 'cap_a(MW)', 'pv_elec(KWh)', 
                                                               'grid_elec(KWh)','butane_cons(tonnes)', 'butane_FARcost(mMAD)',
                                                               'PV_new_cap(MW)','reinv_cap(MW)', 'butane_emissions(MtCO2)',
                                                               'grid_emissions(MtCO2)','Total_emissions(MtCO2)',
                                                               'butane_SUBSIDY(mMAD)','grid_cost(mMAD)','PV_Capex(mMAD)',
                                                               'pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum()

        # Step: Saving Results
        output_folder = os.path.join(pv_folder)
        os.makedirs(output_folder, exist_ok = True)
#         souss_massa1.to_csv(os.path.join(output_folder, 'results.csv'), index=False)
        souss_massa1_summary.to_csv(os.path.join(output_folder, 'summary_results.gz'))

Wall time: 1min 22s


In [87]:
souss_massa1.columns

Index(['Demand point', 'Year', 'province', 'area_m2', 'water_demand(m3)',
       'energy_demand(KWh)', 'pv_elec(KWh)', 'grid_elec(KWh)',
       'butane_cons(tonnes)', 'butane_FARcost(mMAD)', 'butane_ACTcost(mMAD)',
       'butane_SUBSIDY(mMAD)', 'butane_emissions(MtCO2)',
       'grid_emissions(MtCO2)', 'Total_emissions(MtCO2)', 'grid_cost(mMAD)',
       'pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)', 'GWdepth',
       'srad', 'wind', 'cap_m(MW)', 'cap_a(MW)', 'pv_cc', 'PV_new_cap(MW)',
       'reinv_cap(MW)', 'PV_Capex(mMAD)', 'PV_Opex(mMAD)'],
      dtype='object')

## Comparing scenarios

In [1]:
import pandas as pd
import os

# Reading results files
bau_folder = os.path.join('test dash results', 'Butane_calculations', 'phaseout_None')
late_folder = os.path.join('test dash results', 'Butane_calculations', 'phaseout_2040')
early_folder = os.path.join('test dash results', 'Butane_calculations', 'phaseout_2030')

bau1 = pd.read_csv(os.path.join(bau_folder, '10_PV', 'summary_results.gz'))
# bau = bau.add_prefix('bau_')
bau1['butane_phaseout'] = 'None'
bau1['pv_adoption'] = '10%'
bau2 = pd.read_csv(os.path.join(bau_folder, '20_PV', 'summary_results.gz'))
bau2['butane_phaseout'] = 'None'
bau2['pv_adoption'] = '20%'
bau3 = pd.read_csv(os.path.join(bau_folder, '50_PV', 'summary_results.gz'))
bau3['butane_phaseout'] = 'None'
bau3['pv_adoption'] = '50%'


latePV10 = pd.read_csv(os.path.join(late_folder, '10_PV', 'summary_results.gz'))
# latePV10 = latePV10.add_prefix('LP1_')
latePV10['butane_phaseout'] = 'by 2040'
latePV10['pv_adoption'] = '10%'
latePV20 = pd.read_csv(os.path.join(late_folder, '20_PV', 'summary_results.gz'))
# latePV20 = latePV20.add_prefix('LP2_')
latePV20['butane_phaseout'] = 'by 2040'
latePV20['pv_adoption'] = '20%'
latePV40 = pd.read_csv(os.path.join(late_folder, '50_PV', 'summary_results.gz'))
# latePV20 = latePV20.add_prefix('LP2_')
latePV40['butane_phaseout'] = 'by 2040'
latePV40['pv_adoption'] = '50%'


earlyPV10 = pd.read_csv(os.path.join(early_folder, '10_PV', 'summary_results.gz'))
# earlyPV10 = earlyPV10.add_prefix('EP1_')
earlyPV10['butane_phaseout'] = 'by 2030'
earlyPV10['pv_adoption'] = '10%'
earlyPV20 = pd.read_csv(os.path.join(early_folder, '20_PV', 'summary_results.gz'))
# earlyPV20 = earlyPV20.add_prefix('EP2_')
earlyPV20['butane_phaseout'] = 'by 2030'
earlyPV20['pv_adoption'] = '20%'
earlyPV40 = pd.read_csv(os.path.join(early_folder, '50_PV', 'summary_results.gz'))
# earlyPV20 = earlyPV20.add_prefix('EP2_')
earlyPV40['butane_phaseout'] = 'by 2030'
earlyPV40['pv_adoption'] = '50%'

In [2]:
import plotly.express as px

In [3]:
bau1.columns

Index(['Year', 'water_demand(m3)', 'energy_demand(KWh)', 'cap_a(MW)',
       'pv_elec(KWh)', 'grid_elec(KWh)', 'butane_cons(tonnes)',
       'butane_FARcost(mMAD)', 'PV_new_cap(MW)', 'reinv_cap(MW)',
       'butane_emissions(MtCO2)', 'grid_emissions(MtCO2)',
       'Total_emissions(MtCO2)', 'butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)',
       'PV_Capex(mMAD)', 'pv_demand(KWh)', 'butane_demand(KWh)',
       'grid_demand(KWh)', 'butane_phaseout', 'pv_adoption'],
      dtype='object')

In [5]:
df = latePV10.loc[latePV10.Year>2020]

fig = px.bar(df, x='Year', y='PV_new_cap(MW)', #color='variable',
             title='PV new installed capacity (MW)',
#              color_discrete_sequence=px.colors.qualitative.Set2,
             labels={"PV_new_cap(MW)": "PV new capacity (MW)"}
            )
fig.show()

In [6]:
df = bau2
df.loc[df.Year==2020,'PV_new_cap(MW)'] = 0
df['previous_capacity(MW)'] = df['cap_a(MW)'] - df['PV_new_cap(MW)']
df = df.melt(value_vars=['previous_capacity(MW)', 'PV_new_cap(MW)'], id_vars=['Year'])

fig = px.bar(df, x='Year', y='value', color='variable',
             title='PV new installed capacity (MW)',
#              color_discrete_sequence=px.colors.qualitative.Set2,
             labels={"cap_m(MW)": "PV required capacity (MW)"}
            )
fig.show()

In [7]:
df = bau2.melt(value_vars=['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)'], 
                   id_vars=['Year'])

df.loc[df['variable']=='butane_demand(KWh)', 'variable'] = "Butane"
df.loc[df['variable']=='grid_demand(KWh)', 'variable'] = "Grid"
df.loc[df['variable']=='pv_demand(KWh)', 'variable'] = "PV"

fig = px.bar(df, x='Year',  y='value', color='variable',
             title='Energy demand in agriculture water pumping',
             color_discrete_sequence=px.colors.qualitative.Set2,
             labels={'value': 'Energy demand (kWh)',
                     'variable': ''}
            )
fig.show()

In [8]:
df = bau2.melt(value_vars=['butane_emissions(MtCO2)', 'grid_emissions(MtCO2)'], 
                   id_vars=['Year'])

df.loc[df['variable']=='butane_emissions(MtCO2)', 'variable'] = "Butane"
df.loc[df['variable']=='grid_emissions(MtCO2)', 'variable'] = "Grid"

fig = px.bar(df, x='Year', y='value', color='variable',
             title='Yearly emissions',
#              color_discrete_sequence=px.colors.qualitative.Set2,
             labels={'value': 'Emissions (MtCO2)',
                     'variable': ''}
            )
fig.update_layout(legend=dict(
                            orientation="h",
                            yanchor="bottom",
                            y=-0.2,
                            xanchor="left",
                            x=0
))
fig.show()

In [9]:
#bau.columns
#bau['bau_PV_new_cap(MW)']
# earlyPV20['EP2_reinv_cap(MW)']#-earlyPV20['bau_PV_new_cap(MW)']
earlyPV20.columns

Index(['Year', 'water_demand(m3)', 'energy_demand(KWh)', 'cap_a(MW)',
       'pv_elec(KWh)', 'grid_elec(KWh)', 'butane_cons(tonnes)',
       'butane_FARcost(mMAD)', 'PV_new_cap(MW)', 'reinv_cap(MW)',
       'butane_emissions(MtCO2)', 'grid_emissions(MtCO2)',
       'Total_emissions(MtCO2)', 'butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)',
       'PV_Capex(mMAD)', 'pv_demand(KWh)', 'butane_demand(KWh)',
       'grid_demand(KWh)', 'butane_phaseout', 'pv_adoption'],
      dtype='object')

In [10]:
# all_dfs = pd.concat([bau, latePV10,latePV20,earlyPV10,earlyPV20], axis=1)
# all_dfs.drop(['LP1_Year','LP2_Year','EP1_Year','EP2_Year',
#               'LP1_water_demand(m3)','LP2_water_demand(m3)','EP1_water_demand(m3)','EP2_water_demand(m3)',
#              'LP1_energy_demand(KWh)','LP2_energy_demand(KWh)','EP1_energy_demand(KWh)','EP2_energy_demand(KWh)'], axis=1, inplace=True)
# all_dfs.rename({'bau_Year':'Year',
#                'bau_water_demand(m3)':'water_demand(m3)',
#                'bau_energy_demand(KWh)':'energy_demand(KWh)'}, axis=1, inplace=True)


# conv_fac = 1000000000 # to convert emissions from kgCO2 to Million meteric tons of CO2 MtCO2
# butane_em = 6.67 #kg CO"/Gallon
# gallon2liter = 3.78541 #l/Gallon
# butane_density = 573 #kg/m3
# butane_ef = butane_em / (gallon2liter/1000 * butane_density) #kgCO2/kg
# butane_em_fac = butane_ef * 1000 ##kgCO2/ton
# grid_em_fac = 0.7
# epump_eff_old = 0.45
# epump_eff = 0.45
# butane_req = 1/12.58 #LHV = 12.58 KWh/Kg (amount of butane (kg) required to produce 1 KWh) 

all_dfs = bau1.append([bau2,bau3,latePV10,latePV20,latePV40,earlyPV10,earlyPV20,earlyPV40], ignore_index=True)
# all_dfs['butane_emissions(MtCO2)'] = all_dfs['butane_cons(tonnes)'] * butane_em_fac / conv_fac
# all_dfs['grid_elec(KWh)'] = all_dfs['grid_elec(KWh)']/epump_eff*epump_eff_old
# all_dfs['pv_elec(KWh)'] = all_dfs['pv_elec(KWh)']/epump_eff*epump_eff_old
# all_dfs['grid_emissions(MtCO2)'] = all_dfs['grid_elec(KWh)']*grid_em_fac/conv_fac
# all_dfs['butane_energy(KWh)'] = all_dfs['butane_cons(tonnes)'] / (butane_req/1000) * 0.2 

# all_dfs['grid_cost(mMAD)'] /= 100
# all_dfs['grid_emissions(MtCO2)'] /= 100

In [11]:
# all_dfs['total_energy(KWh)'] = all_dfs[['pv_elec(KWh)','grid_elec(KWh)', 'butane_energy(KWh)']].sum(axis=1)
# all_dfs['pv_share'] = all_dfs['pv_elec(KWh)'] / all_dfs['total_energy(KWh)']
# all_dfs['grid_share'] = all_dfs['grid_elec(KWh)'] / all_dfs['total_energy(KWh)']
# all_dfs['butane_share'] = all_dfs['butane_energy(KWh)'] / all_dfs['total_energy(KWh)']
# all_dfs

In [12]:
all_dfs.columns

Index(['Year', 'water_demand(m3)', 'energy_demand(KWh)', 'cap_a(MW)',
       'pv_elec(KWh)', 'grid_elec(KWh)', 'butane_cons(tonnes)',
       'butane_FARcost(mMAD)', 'PV_new_cap(MW)', 'reinv_cap(MW)',
       'butane_emissions(MtCO2)', 'grid_emissions(MtCO2)',
       'Total_emissions(MtCO2)', 'butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)',
       'PV_Capex(mMAD)', 'pv_demand(KWh)', 'butane_demand(KWh)',
       'grid_demand(KWh)', 'butane_phaseout', 'pv_adoption',
       'previous_capacity(MW)'],
      dtype='object')

In [13]:
df = all_dfs.loc[all_dfs.Year >= 2021].groupby(['butane_phaseout', 'pv_adoption'])[['water_demand(m3)', 'energy_demand(KWh)', 'pv_elec(KWh)',
                                              'grid_elec(KWh)', 'butane_cons(tonnes)', 'butane_FARcost(mMAD)',
                                              'PV_new_cap(MW)', 'reinv_cap(MW)', 'butane_emissions(MtCO2)',
                                              'grid_emissions(MtCO2)', 'Total_emissions(MtCO2)',
                                              'butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)', 'PV_Capex(mMAD)',
                                              'pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum().reset_index()

In [14]:
dff = df.copy().melt(value_vars=['butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)', 'PV_Capex(mMAD)'], 
         id_vars=['butane_phaseout', 'pv_adoption'])

dff.loc[dff['variable']=="butane_SUBSIDY(mMAD)", 'variable'] = "Butane"
dff.loc[dff['variable']=="grid_cost(mMAD)", 'variable'] = "Grid"
dff.loc[dff['variable']=="PV_Capex(mMAD)", 'variable'] = "PV"

fig = px.bar(dff, x='butane_phaseout', y='value', color='variable',
             facet_col='pv_adoption', title='Total cost under different butane phaseout scenarios and PV adoption',
             color_discrete_sequence=px.colors.qualitative.Set2,
             labels={
                     "butane_phaseout": "Butane phase-out",
                     "pv_adoption": "PV adoption by 2040",
                     "value": "Total costs (mMAD)",
                     "variable": 'Source'
                     }
            )
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()
# fig.write_image('TotalCosts.pdf', width=900)

In [65]:
all_dfs['Total_emissions(MtCO2)'] = all_dfs[['butane_emissions(MtCO2)', 'grid_emissions(MtCO2)']].sum(axis=1)
dff = all_dfs.groupby(['Year','butane_phaseout', 'pv_adoption'])['Total_emissions(MtCO2)'].sum().reset_index()

dff = dff.copy().melt(value_vars=['Total_emissions(MtCO2)'], 
         id_vars=['Year','butane_phaseout', 'pv_adoption'])

fig = px.line(dff, x='Year', y='value', color='butane_phaseout',
             facet_col='pv_adoption', title='Annual emissions under different butane phaseout scenarios and PV adoption',
             color_discrete_sequence=px.colors.qualitative.T10,
             labels={
                     "butane_phaseout": "Butane phase-out",
                     "pv_adoption": "PV adoption by 2040",
                     "value": "Emissions (MtCO2)"
                     },
#              facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
             facet_col_spacing=0.06,
             )
fig.show()
# fig.write_image('AnnualEmissions.pdf', width=900)

In [66]:
dff = df.copy().melt(value_vars=['butane_emissions(MtCO2)', 'grid_emissions(MtCO2)'], 
         id_vars=['butane_phaseout', 'pv_adoption'])

dff.loc[dff['variable']=="butane_emissions(MtCO2)", 'variable'] = "Butane"
dff.loc[dff['variable']=="grid_emissions(MtCO2)", 'variable'] = "Grid"

fig = px.bar(dff, x='butane_phaseout', y='value', color='variable',
             facet_col='pv_adoption', title='Total emissions under different butane phaseout scenarios and PV adoption',
             color_discrete_sequence=px.colors.qualitative.Vivid,
             labels={
                     "butane_phaseout": "Butane phase-out",
                     "pv_adoption": "PV adoption by 2040",
                     "value": "Emissions (MtCO2)",
                     "variable": 'Source'
                     }
            )

fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':['None','by 2040','by 2030']})

fig.show()
# fig.write_image('TotalEmissions.pdf', width=900)

In [67]:
df = all_dfs.loc[(all_dfs.Year >= 2021) & (all_dfs.Year < 2050)].copy()

df['Total_emissions(MtCO2)'] = df[['butane_emissions(MtCO2)', 'grid_emissions(MtCO2)']].sum(axis=1)
df['Total_costs(mMAD)'] = df[['butane_SUBSIDY(mMAD)', 'grid_cost(mMAD)', 'PV_Capex(mMAD)']].sum(axis=1)

dff = pd.DataFrame()
for key, group in df.groupby(['butane_phaseout', 'pv_adoption']):
#     print(key)
    _dff = group.copy()
    
    _dff['Cumulative emissions (MtCO2)'] = _dff['Total_emissions(MtCO2)'].cumsum()
    _dff['Cumulative costs (mMAD)'] = _dff['Total_costs(mMAD)'].cumsum()
#     print(_dff)
    dff = dff.append(_dff, ignore_index=True)

# dff['Cumulative emissions (MtCO2)'] = dff['Total_emissions(MtCO2)'].cumsum()
# dff['Cumulative costs (mMAD)'] = dff['Total_costs(mMAD)'].cumsum()

pv_adop_dic = {'10%': 0.1, '20%': 0.2, '50%': 0.5}

dff = dff.groupby(['butane_phaseout', 'pv_adoption']).sum().reset_index()

dff['pv_adoption_number'] = [pv_adop_dic[i] for i in dff['pv_adoption']]

# dff = dff.melt(value_vars=['Total_emissions(MtCO2)', 'Total_costs(mMAD)'], 
#          id_vars=['butane_phaseout', 'pv_adoption'])

fig = px.scatter(dff, x='Total_costs(mMAD)', y='Total_emissions(MtCO2)',
                 title='Total emissions and costs under different butane phaseout scenarios and PV adoption', 
                 color='butane_phaseout', size='pv_adoption_number',
#              animation_frame="Year", animation_group="pv_adoption", hover_name="pv_adoption",
#              range_x=[dff['pv_elec(KWh)'].min(),dff['pv_elec(KWh)'].max()], size_max=55,
#              range_y=[dff['Total_emissions(MtCO2)'].min(),dff['Total_emissions(MtCO2)'].max()],
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 labels={
                         "butane_phaseout": "Butane phase-out",
                         "pv_adoption": "PV adoption share by 2040",
                         "Total_emissions(MtCO2)": "Emissions (MtCO2)",
                         "Total_costs(mMAD)": 'Total costs (mMAD)',
                         "pv_adoption_number": 'PV share by 2040'
                         }
                )

fig.show()
# fig.write_image('EmissionsVsCosts.pdf', width=900)

In [68]:
dff = all_dfs.groupby(['Year','butane_phaseout', 'pv_adoption'])[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum().reset_index()

dff['pv_demand(%)'] = dff['pv_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)
dff['butane_demand(%)'] = dff['butane_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)
dff['grid_demand(%)'] = dff['grid_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)

dff = dff.melt(value_vars=['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)'], 
         id_vars=['Year','butane_phaseout', 'pv_adoption'])

dff.loc[dff['variable']=="butane_demand(KWh)", 'variable'] = "Butane"
dff.loc[dff['variable']=="grid_demand(KWh)", 'variable'] = "Grid"
dff.loc[dff['variable']=="pv_demand(KWh)", 'variable'] = "PV"
dff['value'] /= 1000000

fig = px.bar(dff, x='Year', y='value', color='variable',
             facet_col='pv_adoption', facet_row='butane_phaseout', title='Energy sources under different butane phaseout scenarios and PV adoption',
             color_discrete_sequence=px.colors.qualitative.Set2,
             labels={
                     "butane_phaseout": "phaseout",
                     "pv_adoption": "PV adoption by 2040",
                     "value": "Energy (GWh)",
                     "variable": 'Source'
                     },
             facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
             facet_col_spacing=0.04,
            )
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()
# fig.write_image('EnergySources.pdf', width=900, height=600)

In [69]:
dff = all_dfs.groupby(['Year','butane_phaseout', 'pv_adoption'])[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum().reset_index()

dff['pv_demand(%)'] = dff['pv_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)
dff['butane_demand(%)'] = dff['butane_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)
dff['grid_demand(%)'] = dff['grid_demand(KWh)'] / dff[['pv_demand(KWh)', 'butane_demand(KWh)', 'grid_demand(KWh)']].sum(axis=1)

dff = dff.melt(value_vars=['pv_demand(%)', 'butane_demand(%)', 'grid_demand(%)'], 
         id_vars=['Year','butane_phaseout', 'pv_adoption'])

dff.loc[dff['variable']=="butane_demand(%)", 'variable'] = "Butane"
dff.loc[dff['variable']=="grid_demand(%)", 'variable'] = "Grid"
dff.loc[dff['variable']=="pv_demand(%)", 'variable'] = "PV"

fig = px.bar(dff, x='Year', y='value', color='variable',
             facet_col='pv_adoption', facet_row='butane_phaseout', title='Share of energy sources under different butane phaseout scenarios and PV adoption',
             color_discrete_sequence=px.colors.qualitative.Set2,
             labels={
                     "butane_phaseout": "phaseout",
                     "pv_adoption": "PV adoption by 2040",
                     "value": "Energy share",
                     "variable": 'Source'
                     },
             category_orders={'butane_phaseout': ['None', 'by 2040', 'by 2030']},
             facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
             facet_col_spacing=0.04,
            )
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()
# fig.write_image('EnergyShares.pdf', width=900, height=600)

In [10]:
all_dfs.sum()

Year                                                                  378510
water_demand(m3)                                                 1.31544e+11
energy_demand(KWh)                                               7.91624e+10
pv_elec(KWh)                                                     2.35182e+10
grid_elec(KWh)                                                   1.34917e+11
butane_cons(tonnes)                                              3.12665e+06
butane_FARcost(mMAD)                                                 10422.2
PV_new_cap(MW)                                                       2248.59
reinv_cap(MW)                                                        1491.85
butane_emissions(MtCO2)                                             0.662225
grid_emissions(MtCO2)                                                2.37049
Total_emissions(MtCO2)                                               3.03271
butane_SUBSIDY(mMAD)                                                 20844.4

In [11]:
all_dfs

,Year,water_demand(m3),energy_demand(KWh),pv_elec(KWh),grid_elec(KWh),butane_cons(tonnes),butane_FARcost(mMAD),PV_new_cap(MW),reinv_cap(MW),butane_emissions(MtCO2),grid_emissions(MtCO2),Total_emissions(MtCO2),butane_SUBSIDY(mMAD),grid_cost(mMAD),PV_Capex(mMAD),butane_phaseout,pv_adoption
0,2020,7.479313e+08,3.503796e+08,7.786213e+07,5.450349e+08,27852.112070,92.840374,93.592417,0.000000,0.005899,0.009576,0.015475,185.680747,545.034886,655.146918,None,Low
1,2021,7.118628e+08,4.118799e+08,9.152887e+07,6.407021e+08,32740.852396,109.136175,0.080980,0.000000,0.006935,0.011257,0.018192,218.272349,640.702103,0.565497,None,Low
2,2022,7.201413e+08,4.009355e+08,8.909678e+07,6.236774e+08,31870.865706,106.236219,14.638226,0.000000,0.006750,0.010958,0.017708,212.472438,623.677430,101.757138,None,Low
3,2023,6.442514e+08,4.298114e+08,9.551365e+07,6.685955e+08,34166.249796,113.887499,0.075571,0.000000,0.007236,0.011747,0.018984,227.774999,668.595546,0.521805,None,Low
4,2024,6.610189e+08,4.181049e+08,9.291220e+07,6.503854e+08,33235.684045,110.785613,1.887994,0.000000,0.007039,0.011427,0.018467,221.571227,650.385408,12.919921,None,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,2046,7.937677e+08,4.637520e+08,2.061120e+08,8.244480e+08,0.000000,0.000000,5.929948,0.132203,0.000000,0.014486,0.014486,0.000000,824.448011,21.319373,Early,High
182,2047,8.019210e+08,4.557931e+08,2.025747e+08,8.102989e+08,0.000000,0.000000,0.156746,2.934964,0.000000,0.014237,0.014237,0.000000,810.298883,10.682477,Early,High
183,2048,6.778976e+08,4.581649e+08,2.036288e+08,8.145154e+08,0.000000,0.000000,0.154273,19.004605,0.000000,0.014311,0.014311,0.000000,814.515372,65.303674,Early,High
184,2049,6.590121e+08,4.577538e+08,2.034461e+08,8.137845e+08,0.000000,0.000000,6.136654,38.750699,0.000000,0.014298,0.014298,0.000000,813.784543,151.575614,Early,High


In [14]:
dfcost = all_dfs.reset_index().pivot(index='Year', columns=['water_demand(m3)', 'energy_demand(KWh)', 
                                     'bau_butane_SUBSIDY(mMAD)','LP1_butane_SUBSIDY(mMAD)','LP2_butane_SUBSIDY(mMAD)',
                                     'bau_PV_Capex(mMAD)','LP1_PV_Capex(mMAD)','LP2_PV_Capex(mMAD)'])

KeyError: "None of ['bau_butane_SUBSIDY(mMAD)', 'LP1_butane_SUBSIDY(mMAD)', 'LP2_butane_SUBSIDY(mMAD)', 'bau_PV_Capex(mMAD)', 'LP1_PV_Capex(mMAD)', 'LP2_PV_Capex(mMAD)'] are in the columns"

In [183]:
costs = pd.DataFrame({'Year':bau['Year'],
                       'Ref-Subs(mMAD)':bau['butane_SUBSIDY(mMAD)'],
                       'LatePV10-Subs(mMAD)':latePV10['butane_SUBSIDY(mMAD)'],
                        'LatePV20-Subs(mMAD)':latePV20['butane_SUBSIDY(mMAD)'],
                        'EarlyPV10-Subs(mMAD)':earlyPV10['butane_SUBSIDY(mMAD)'],
                        'EarlyPV20-Subs(mMAD)':earlyPV20['butane_SUBSIDY(mMAD)'],
                        'Ref-PVInv(mMAD)':bau['PV_Capex(mMAD)'],
                       'LatePV10-PVInv(mMAD)':latePV10['PV_Capex(mMAD)'],
                        'LatePV20-PVInv(mMAD)':latePV20['PV_Capex(mMAD)'],
                        'EarlyPV10-PVInv(mMAD)':earlyPV10['PV_Capex(mMAD)'],
                        'EarlyPV20-PVInv(mMAD)':earlyPV20['PV_Capex(mMAD)'],
                     'Ref-PVcap(MW)':(bau['PV_new_cap(MW)']+bau['reinv_cap(MW)']),
                       'LatePV10-PVcap(MW)':(latePV10['PV_new_cap(MW)']+latePV10['reinv_cap(MW)']),
                        'LatePV20-PVcap(MW)':(latePV20['PV_new_cap(MW)']+latePV20['reinv_cap(MW)']),
                        'EarlyPV10-PVcap(MW)':(earlyPV10['PV_new_cap(MW)']+earlyPV10['reinv_cap(MW)']),
                        'EarlyPV20-PVcap(MW)':(earlyPV20['PV_new_cap(MW)']+earlyPV20['reinv_cap(MW)'])})

costs['Ref-Savings'] = costs['Ref-Subs(mMAD)']-costs['Ref-PVInv(mMAD)']
costs['LatePV10-Savings'] = costs['LatePV10-Subs(mMAD)']-costs['LatePV10-PVInv(mMAD)']
costs['LatePV20-Savings'] = costs['LatePV20-Subs(mMAD)']-costs['LatePV20-PVInv(mMAD)']
costs['EarlyPV10-Savings'] = costs['EarlyPV10-Subs(mMAD)']-costs['EarlyPV10-PVInv(mMAD)']
costs['EarlyPV20-Savings'] = costs['EarlyPV20-Subs(mMAD)']-costs['EarlyPV20-PVInv(mMAD)']

In [184]:
#costs1= costs.loc[costs['Year'] < 2041]
costs.to_csv('cost_summary_all.csv')

## Baseline Scenario

No phaseout of Butane. The distribution of all technologies (Butane, PV and Grid) will continue to be the same as the current distribution which is: 

* PV = 10% 
* Butane = 20%
* Grid = 70% 

In [93]:
year = 2020
#df = bau.df.loc[(bau.df.Year==year)].copy()
df.loc[df['type'].str.contains('DS'), 'type'] = 'Desalinated water'
df.loc[df['type'].str.contains('Pipeline'), 'type'] = 'Conveyance'
df.loc[df['type'].str.contains('Pipeline'), 'type'] = 'Conveyance'


df_group = df.groupby(['Province','type'])[['sswd', 'swpa_e']].sum()/1000000
df_group.rename(columns={'sswd': 'Water demand [Mm3]', 'swpa_e': 'Energy demand [GWh]'}, inplace=True)
df_group['Energy intensity [kWh/m3]'] = df_group['Energy demand [GWh]'] / df_group['Water demand [Mm3]']
# df_group.reset_index(inplace=True)
# df_group.to_csv('Sumary for Ref scenario with Desal and no CC.csv')
df_group

KeyError: 'type'

### Step 1: Technologies distribution calculations

In [7]:
df = cropland.copy()

In [ ]:
# Original code: 
pv_change = [0] #the negative sign means increase
butane_change = [0]
for pv_share in pv_change:
    for butane_share in butane_change:
        pv_share1 = 0.1
        butane_share1 = 0.2
        df.start_year = 2020
        df.end_year = 2050
        bt_rates = {2020: 0, 2030: butane_share*0.5, 2040: butane_share*0.2, 2050: butane_share}
        pv_rates = {2020: 0, 2030: pv_share*0.5, 2040: pv_share*0.2, 2050: pv_share}
        butane_dist = create_tech_dist(df.start_year, df.end_year, butane_share1, bt_rates, 'polynomial')
        pv_dist = create_tech_dist(df.start_year, df.end_year, pv_share1, pv_rates, 'polynomial')
        grid_dist = (1-(butane_dist+pv_dist))

In [184]:
#This function can be moved to the nexus tool later:

def create_tech_dist(iyear, eyear, share1, rates, method, order=2):
    tech_dist = pd.DataFrame({'Year': range(iyear, eyear+1)})
    if method!='step':
        for year, rate in rates.items():
            tech_dist.loc[tech_dist.Year==year, 'rate'] = rate
        tech_dist['share'] = (1-tech_dist['rate']) * share1
        tech_dist['share'] = tech_dist.share.interpolate(method=method, order=order)
    else:
        for year, rate in rates.items():
            tech_dist.loc[tech_dist.Year>=year, 'rate'] = rate
        tech_dist['share'] = (1-tech_dist['rate']) * share1
    return tech_dist.set_index('Year').share

In [189]:
#copy1
scenarios = ['bau','late_po','early_po']
pv_levels = ['low_PV', 'high_PV']

for scenario in scenarios:
    for pv_level in pv_levels:
        
        butane_change ={'bau':0,'late_po':1,'early_po':1 }
        butane_share = butane_change[scenario]
        
        end_year ={'bau':2040,'late_po':2040,'early_po':2040 }
        year = end_year[scenario]

        pv_change ={'low_PV':0,'high_PV':1}
        pv_share = pv_change[pv_level]


        pv_change = [-0.3] #the negative sign means increase
        butane_change = [1]
        for pv_share in pv_change:
            for butane_share in butane_change:
                pv_share1 = 0.1
                butane_share1 = 0.2
                df.start_year = 2020
                df.mid_year = ((df.start_year+df.end_year)/2)
                df.end_year = year
                
                bt_rates = {df.start_year: 0, df.mid_year: butane_share*0.5, df.end_year: butane_share}
                pv_rates = {df.start_year: 0, df.mid_year: pv_share*0.5, df.end_year: pv_share}
                
                butane_dist = create_tech_dist(df.start_year, df.end_year, butane_share1, bt_rates, 'polynomial')
                pv_dist = create_tech_dist(df.start_year, df.end_year, pv_share1, pv_rates, 'polynomial')

        

In [190]:
#copy2
scenarios = ['bau','late_po','early_po']
pv_levels = ['low_PV', 'high_PV']

for scenario in scenarios:
    for pv_level in pv_levels:
        
        butane_change ={'bau':0,'late_po':1,'early_po':1 }
        butane_share = butane_change[scenario]
        
        end_year ={'bau':2050,'late_po':2040,'early_po':2040 }
        year = end_year[scenario]

        pv_change ={'low_PV':0,'high_PV':1}
        pv_share = pv_change[pv_level]
        
        pv_change = [-0.3] #the negative sign means increase
        butane_change = [1]
        for pv_share in pv_change:
            for butane_share in butane_change:
                pv_share1 = 0.1
                butane_share1 = 0.2
                df.start_year = 2020
                df.end_year = 2050
                
                bt_bau = {2020: 0, 2025: 0, 2030: 0, 2040: 0, 2050: 0}
                bt_early = {2020: 0, 2025: butane_share*0.7, 2030: butane_share, 2040: butane_share*1.1, 2050: butane_share*1.2}
                bt_late = {2020: 0, 2030: butane_share*0.5, 2040: butane_share, 2050: butane_share*1.2}
                
                pv_rates = {2020: 0, 2025: pv_share*0.7, 2030: pv_share*0.5, 2040: pv_share*0.3, 2050: pv_share}
                
                bt_rates ={'bau':bt_bau,'late_po':bt_late,'early_po':bt_early }
                bt_rate=bt_rates[scenario]
                
                butane_dist = create_tech_dist(df.start_year, df.end_year, butane_share1, bt_rates, 'polynomial')
                pv_dist = create_tech_dist(df.start_year, df.end_year, pv_share1, pv_rates, 'polynomial')

 

ValueError: Must have equal len keys and value when setting with an iterable

In [179]:
def create_learning_curve(iyear, eyear, cc, rates, method, order=2):
    learning_curve = pd.DataFrame({'Year': range(iyear, eyear+1)})
    if method!='step':
        for year, rate in rates.items():
            learning_curve.loc[learning_curve.Year==year, 'rate'] = rate
        learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
        learning_curve['capital_cost'] = learning_curve.capital_cost.interpolate(method=method, order=order)
    else:
        for year, rate in rates.items():
            learning_curve.loc[learning_curve.Year>=year, 'rate'] = rate
        learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
    return learning_curve.set_index('Year').capital_cost

In [180]:
# The capital cost of PV in Morocco is 7 MAD/ Wp  (excluding tax) = 7000 (MAD/KW) or = 7 (mMAD/MW)

pv_rate = 0.3 #This assumes 30% drop in PV CAPEX by 2040. 
cc = 7
pv_cf = 0.25
pv_life=15
df.start_year=2020
df.end_year=2050
#capital_cost = 1000
rates = {2020: 0, 2030: pv_rate*0.3, 2040: pv_rate, 2050: pv_rate*1.3}
cc_solar_curve = create_learning_curve(df.start_year, df.end_year, cc, rates, 'polynomial')
om_cost=0.01 
efficiency=1

In [181]:
dist = pd.DataFrame({'butane_share':pd.Series(butane_dist),
                   'pv_share':pd.Series(pv_dist),
                   'grid_share':pd.Series(grid_dist),
                    'pv_cc':pd.Series(cc_solar_curve)})

dist['butane_share'].fillna(method='ffill',inplace=True)
dist['pv_share'].fillna(method='ffill',inplace=True)

dist.loc[dist['butane_share']<0, 'butane_share'] =0
dist.loc[dist['pv_share']<0, 'pv_share'] =0

dist['grid_share']= 1-(dist['pv_share']+dist['butane_share'])

dist
#Check how to set the end year for phaseout to 2040 and take the same distibution for other years. 

,butane_share,pv_share,grid_share,pv_cc
Year,,,,
2020,0.20000,0.100000,0.700000,7.0000
2021,0.19045,0.106721,0.702829,6.9874
2022,0.18080,0.112182,0.707018,6.9636
2023,0.17105,0.116382,0.712568,6.9286
2024,0.16120,0.119321,0.719479,6.8824
2025,0.15125,0.121000,0.727750,6.8250
2026,0.14120,0.121418,0.737382,6.7564
2027,0.13105,0.120576,0.748374,6.6766
2028,0.12080,0.118654,0.760546,6.5856


In [107]:
bau = pd.merge(cropland, dist, on='Year')

### Step 2: Butane calculations

In [37]:
butane_req = 1/12.58 # LHV = 12.58 KWh/Kg (amount of butane (kg) required to produce 1 KWh) 
butane_em_fac = 211.8 # Butane emission factor = 211.77 kg CO2 per tons
bpump_eff = 0.2 #efficiency of butane 
epump_eff = 0.45 #assumption
conv_fac = 1000000000 # to convert emissions from kgCO2 to Million meteric tons of CO2 MtCO2

bau['pv_load(KW)'] = bau['peak_load(KW)']*bau['pv_share']
bau['pv_demand(KWh)'] = bau['energy_demand(KWh)']*bau['pv_share']
bau['butane_demand(KWh)'] = bau['energy_demand(KWh)']*bau['butane_share']
bau['grid_demand(KWh)'] = bau['energy_demand(KWh)']*bau['grid_share']


bau['pv_elec(KWh)'] = bau['pv_demand(KWh)']/epump_eff
bau['grid_elec(KWh)'] = bau['grid_demand(KWh)']/epump_eff

bau['butane_cons(kg)'] = (bau['butane_demand(KWh)']*butane_req)/bpump_eff
bau['butane_cons(tonnes)'] = bau['butane_cons(kg)']/1000
bau['butane_emissions(MtCO2)'] = bau['butane_cons(tonnes)'] * butane_em_fac/conv_fac

bau['butane_FARcost(mMAD)'] = (bau['butane_cons(kg)']*(40/12))/1000000 #in million MAD, this is what farmers pay
bau['butane_ACTcost(mMAD)'] = (bau['butane_cons(kg)']*(120/12))/1000000 #in million MAD, this is what farmers pay
bau['butane_SUBSIDY(mMAD)'] = (bau['butane_cons(kg)']*(80/12))/1000000 #in million MAD, this is the total subsidy cost



### Step 3: Grid calculations

In [38]:
# Estimating the electriicty from the grid and emissions
grid_em_fac = 1.757 #kgco2/kwh  , This is based on data from IEA, Morocco 2019 
grid_cost =100 # MAD/KWh, Assumption and to be updated 

#scenario1['grid_elec (kwh)'] = scenario1['grid_demand_kwh']/epump_eff #already calculated above
bau['grid_emissions(MtCO2)'] = bau['grid_elec(KWh)']*grid_em_fac/conv_fac
bau['grid_cost(mMAD)'] = (bau['grid_elec(KWh)']*grid_cost)/1000000
bau['Total_emissions(MtCO2)'] = (bau['grid_emissions(MtCO2)'] + bau['butane_emissions(MtCO2)'])

### Step 4: Solar PV  calculations

In [44]:
# Estimating the required monthly capacity of pv NOTE: 1 kWh = 3600 kJ

bau['cf'] = bau['srad'] / (24*60*60) #This will give the cf in solar rad: (kJ/(m2.day))*30.day/month*1h/(24h*60m*60s) =kWh/(m2.month)*30/(60*60)
bau['cap_m(MW)'] = bau['pv_load(KW)'] / bau['cf']/1000   #to convert to MW, check the units


In [48]:
bau1 = bau.groupby(['Demand point','Year'])[['water_demand(m3)','energy_demand(KWh)', 'pv_elec(KWh)', 'grid_elec(KWh)','butane_cons(tonnes)', 'butane_FARcost(mMAD)', 'butane_ACTcost(mMAD)','butane_SUBSIDY(mMAD)','butane_emissions(MtCO2)','grid_emissions(MtCO2)','Total_emissions(MtCO2)','grid_cost(mMAD)']].sum()

bau1['GWdepth'] = (bau.groupby(['Demand point','Year'])['wtd'].mean())
bau1['srad'] = (bau.groupby(['Demand point','Year'])['srad'].mean())
bau1['wind'] = (bau.groupby(['Demand point','Year'])['wind'].mean())
bau1['cap_m(MW)'] = (bau.groupby(['Demand point','Year'])['cap_m(MW)'].mean())
bau1['cap_a(MW)'] = (bau.groupby(['Demand point','Year'])['cap_m(MW)'].max())
bau1['pv_cc'] = (bau.groupby(['Demand point','Year'])['pv_cc'].mean())


In [50]:
bau1.reset_index(inplace=True)
bau1.loc[bau1['Year']==2020, 'PV_new_cap(MW)'] = bau1['cap_a(MW)']
bau1.loc[bau1['Year']!=2020, 'PV_new_cap(MW)'] = bau1['cap_a(MW)'] - bau1['cap_a(MW)'].shift(1)
bau1.loc[bau1['PV_new_cap(MW)']<0, 'PV_new_cap(MW)'] =0
bau1['reinv_cap(MW)'] = bau1['cap_a(MW)'].shift(pv_life).fillna(0)
bau1['PV_Capex(mMAD)']=(bau1['PV_new_cap(MW)']+bau1['reinv_cap(MW)'])*bau1['pv_cc']
bau1['PV_Opex(mMAD)']=(bau1['PV_Capex(mMAD)']*om_cost)

In [51]:
bau1_summary = bau1.groupby(['Year'])[['water_demand(m3)','energy_demand(KWh)', 'pv_elec(KWh)', 'grid_elec(KWh)','butane_cons(tonnes)', 'butane_FARcost(mMAD)','PV_new_cap(MW)', 'butane_emissions(MtCO2)','grid_emissions(MtCO2)','Total_emissions(MtCO2)','butane_SUBSIDY(mMAD)','grid_cost(mMAD)','PV_Capex(mMAD)']].sum()

In [52]:
bau1_summary.head()

,water_demand(m3),energy_demand(KWh),pv_elec(KWh),grid_elec(KWh),butane_cons(tonnes),butane_FARcost(mMAD),PV_new_cap(MW),butane_emissions(MtCO2),grid_emissions(MtCO2),Total_emissions(MtCO2),butane_SUBSIDY(mMAD),grid_cost(mMAD),PV_Capex(mMAD)
Year,,,,,,,,,,,,,
2020,7.479313e+08,3.503796e+08,7.786213e+07,5.450349e+08,27852.112070,92.840374,93.592417,0.005899,0.957626,0.963525,185.680747,54503.488643,1436.212053
2021,7.118628e+08,4.118799e+08,9.152887e+07,6.407021e+08,32740.852396,109.136175,0.080980,0.006935,1.125714,1.132648,218.272349,64070.210266,782.865444
2022,7.201413e+08,4.009355e+08,8.909678e+07,6.236774e+08,31870.865706,106.236219,14.638226,0.006750,1.095801,1.102551,212.472438,62367.742979,898.672770
2023,6.442514e+08,4.298114e+08,9.551365e+07,6.685955e+08,34166.249796,113.887499,0.075571,0.007236,1.174722,1.181959,227.774999,66859.554601,795.010274
2024,6.610189e+08,4.181049e+08,9.291220e+07,6.503854e+08,33235.684045,110.785613,1.887994,0.007039,1.142727,1.149766,221.571227,65038.540823,609.778120


## Phaseout scenarios

In [21]:
df2=df3=cropland.copy()

### Step 1: Technologies distribution calculations

In [22]:
#This function can be moved to the nexus tool later:

def create_tech_dist(iyear, eyear, share1, rates, method, order=2):
    tech_dist = pd.DataFrame({'Year': range(iyear, eyear+1)})
    if method!='step':
        for year, rate in rates.items():
            tech_dist.loc[tech_dist.Year==year, 'rate'] = rate
        tech_dist['share'] = (1-tech_dist['rate']) * share1
        tech_dist['share'] = tech_dist.share.interpolate(method=method, order=order)
    else:
        for year, rate in rates.items():
            tech_dist.loc[tech_dist.Year>=year, 'rate'] = rate
        tech_dist['share'] = (1-tech_dist['rate']) * share1
    return tech_dist.set_index('Year').share

In [23]:
pv_change = [-0.8] #the negative sign means increase
butane_change = [1]
for pv_share in pv_change:
    for butane_share in butane_change:
        pv_share1 = 0.1
        butane_share1 = 0.2
        df2.start_year = 2020
        df2.end_year = 2030
        bt_rates = {2020: 0, 2025: butane_share*0.5, 2030: butane_share}
        pv_rates = {2020: 0, 2030: pv_share*0.5, 2040: pv_share*0.2, 2050: pv_share}
        butane_dist = create_tech_dist(df.start_year, df.end_year, butane_share1, bt_rates, 'polynomial')
        pv_dist = create_tech_dist(df.start_year, df.end_year, pv_share1, pv_rates, 'polynomial')
        
        

In [24]:
def create_learning_curve(iyear, eyear, cc, rates, method, order=2):
    learning_curve = pd.DataFrame({'Year': range(iyear, eyear+1)})
    if method!='step':
        for year, rate in rates.items():
            learning_curve.loc[learning_curve.Year==year, 'rate'] = rate
        learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
        learning_curve['capital_cost'] = learning_curve.capital_cost.interpolate(method=method, order=order)
    else:
        for year, rate in rates.items():
            learning_curve.loc[learning_curve.Year>=year, 'rate'] = rate
        learning_curve['capital_cost'] = (1-learning_curve['rate']) * cc
    return learning_curve.set_index('Year').capital_cost

In [25]:
pv_rate = 0.3
cc = 1000
pv_cf = 0.25
df3.start_year=2020
df3.end_year=2050
#capital_cost = 1000
rates = {2020: 0, 2030: pv_rate*0.3, 2040: pv_rate, 2050: pv_rate*1.3}
cc_solar_curve = create_learning_curve(df3.start_year, df3.end_year, cc, rates, 'polynomial')
om_cost=0.01 
efficiency=1

In [26]:
dist = pd.DataFrame({'butane_share':pd.Series(butane_dist),
                   'pv_share':pd.Series(pv_dist),
                   'pv_cc':pd.Series(cc_solar_curve)})

dist.reset_index(inplace=True)

dist['butane_share'].fillna(method='ffill',inplace=True)
dist['grid_share']= 1-(dist['pv_share']+dist['butane_share'])

dist


,Year,butane_share,pv_share,pv_cc,grid_share
0,2020,0.20,0.100000,1000.0,0.700000
1,2021,0.18,0.108020,998.2,0.711980
2,2022,0.16,0.115147,994.8,0.724853
3,2023,0.14,0.121380,989.8,0.738620
4,2024,0.12,0.126720,983.2,0.753280
5,2025,0.10,0.131167,975.0,0.768833
6,2026,0.08,0.134720,965.2,0.785280
7,2027,0.06,0.137380,953.8,0.802620
8,2028,0.04,0.139147,940.8,0.820853
9,2029,0.02,0.140020,926.2,0.839980


In [55]:
scenario1 = pd.merge(cropland, dist, on='Year')

In [56]:
scenario1['pv_share'].sum()

369589.2480000002

In [57]:
scenario1['grid_share'].sum()

2108815.872000002

In [58]:
threshold = 100
#mylist = scenario1[scenario1['wtd'] > threshold]['']
scenario1.loc[scenario1['wtd']>threshold, 'pv_share'] =0
scenario1['grid_share'] = (1-(scenario1['butane_share']+scenario1['pv_share']))


In [59]:
scenario1['pv_share'].sum()

188452.01760000002

In [60]:
scenario1['grid_share'].sum()

2289953.1024000016

### Step 2: Butane calculations

In [61]:
butane_req = 1/12.58 # LHV = 12.58 KWh/Kg (amount of butane (kg) required to produce 1 KWh) 
bpump_eff = 0.2 #efficiency of butane 
epump_eff = 0.45 #assumption


scenario1['butane_demand_kwh'] = scenario1['energy_demand']*scenario1['butane_share']
scenario1['butane_cons (kg)'] = (scenario1['butane_demand_kwh']*butane_req)/bpump_eff
scenario1['butane_cons (tonnes)'] = scenario1['butane_cons (kg)']/1000

scenario1['butane_FARcost (mMAD)'] = (scenario1['butane_cons (kg)']*(40/12))/1000000 #in million MAD, this is what farmers pay
scenario1['butane_ACTcost (mMAD)'] = (scenario1['butane_cons (kg)']*(120/12))/1000000 #in million MAD, this is what farmers pay
scenario1['butane_SUBSIDY (mMAD)'] = (scenario1['butane_cons (kg)']*(80/12))/1000000 #in million MAD, this is the total subsidy cost

### Step 3: Grid calculations

In [62]:
# Estimating the electriicty from the grid and emissions
grid_em_fac = 1.757 #kgco2/kwh  , This is based on data from IEA, Morocco 2019 
grid_cost =100 # MAD/KWh, Assumption and to be updated 
epump_eff = 0.45 #assumption

scenario1['grid_demand_kwh'] = scenario1['energy_demand']*scenario1['grid_share']
scenario1['grid_elec (kwh)'] = scenario1['grid_demand_kwh']/epump_eff


scenario1['grid_emissions'] = scenario1['grid_elec (kwh)']*grid_em_fac
scenario1['grid_cost_mMAD'] = (scenario1['grid_elec (kwh)']*grid_cost)/1000000

In [63]:
scenario1['energy_demand'].sum()/1000000

13193.734697229458

### Step 4: Solar PV calculations

In [67]:
scenario1['pv_demand_kwh'] = scenario1['energy_demand']*scenario1['pv_share']
scenario1['pv_elec (kwh)'] = scenario1['pv_demand_kwh']/epump_eff

In [68]:
# Estimating the required monthly capacity of pv NOTE: 1 kWh = 3600 kJ and 1 day=24 hours
# Solar radiation GIS layer was obtained from WorldClim which gives srad in (kJ/m2.day). 
# To calculate the capacity factor, we will need to convert this to (KWh/m2.hr)

scenario1['cf'] = scenario1['srad'] / (24*3600) #This will give the cf in (KWh/m2.hr) or (KW/m2)
scenario1['cap_m'] = scenario1['pv_elec (kwh)'] / scenario1['cf']/1000   #to convert to MWh, check the units


In [70]:
dfs1 = scenario1.groupby(['Demand point','Year'])[['water_demand','energy_demand', 'pv_elec (kwh)', 'grid_elec (kwh)','butane_cons (tonnes)', 'butane_FARcost (mMAD)', 'butane_ACTcost (mMAD)','butane_SUBSIDY (mMAD)','grid_emissions','grid_cost_mMAD']].sum()

dfs1['GWdepth'] = (scenario1.groupby(['Demand point','Year'])['wtd'].mean())
dfs1['srad'] = (scenario1.groupby(['Demand point','Year'])['srad'].mean())
dfs1['wind'] = (scenario1.groupby(['Demand point','Year'])['wind'].mean())
dfs1['cap_m'] = (scenario1.groupby(['Demand point','Year'])['cap_m'].mean())
dfs1['cap_a'] = (scenario1.groupby(['Demand point','Year'])['cap_m'].max())
dfs1['pv_cc'] = (scenario1.groupby(['Demand point','Year'])['pv_cc'].mean())
dfs1['pv_capex']=dfs1['cap_a']*dfs1['pv_cc']
dfs1['pv_opex']=dfs1['pv_capex']*om_cost


In [73]:
dfs1.reset_index(inplace=True)
dfs1.loc[dfs1['Year']==2020, 'new_cap'] = dfs1['cap_a']
dfs1.loc[dfs1['Year']!=2020, 'new_cap'] = dfs1['cap_a'] - dfs1['cap_a'].shift(1)
dfs1.loc[dfs1['new_cap']<0, 'new_cap'] =0

In [75]:
dfs1['reinv_cap'] = dfs1['cap_a'].shift(pv_life).fillna(0)
dfs1['pv_capex']=(dfs1['new_cap']+dfs1['reinv_cap'])*dfs1['pv_cc']

In [77]:
scenario1_summary = dfs1.groupby(['Year'])[['water_demand','energy_demand','butane_cons (tonnes)', 'butane_FARcost (mMAD)', 'butane_ACTcost (mMAD)','butane_SUBSIDY (mMAD)','pv_capex','pv_opex','grid_emissions','grid_cost_mMAD']].sum()

In [78]:
scenario1_summary.head()

,water_demand,energy_demand,butane_cons (tonnes),butane_FARcost (mMAD),butane_ACTcost (mMAD),butane_SUBSIDY (mMAD),pv_capex,pv_opex,grid_emissions,grid_cost_mMAD
Year,,,,,,,,,,
2020,7.479313e+08,3.503796e+08,27852.112070,92.840374,278.521121,185.680747,5.433053e+07,221099.499979,1.044838e+09,59467.161217
2021,7.118628e+08,4.118799e+08,29458.581943,98.195273,294.585819,196.390546,3.170230e+07,206902.804774,1.256252e+09,71499.807670
2022,7.201413e+08,4.009355e+08,25836.648466,86.122162,258.366485,172.244323,3.650397e+07,257232.450748,1.246688e+09,70955.493474
2023,6.442514e+08,4.298114e+08,25035.319538,83.451065,250.353195,166.902130,3.131888e+07,219823.803140,1.359108e+09,77353.917348
2024,6.610189e+08,4.181049e+08,22134.965574,73.783219,221.349656,147.566437,2.557655e+07,233435.159601,1.340221e+09,76278.933548


In [80]:
butane_subsidy_change = bau1_summary['butane_SUBSIDY (mMAD)'].sum() - scenario1_summary['butane_SUBSIDY (mMAD)'].sum()
pv_investment_change = bau1_summary['pv_capex'].sum() - scenario1_summary['pv_capex'].sum()
grid_emissions_change = bau1_summary['grid_emissions'].sum() - scenario1_summary['grid_emissions'].sum()

In [81]:
butane_subsidy_change

2735.2276136112014

In [82]:
pv_investment_change

938892459.6971364

In [83]:
grid_emissions_change

-6723255823.795586

## This is how the summary table may look like


summary = 
Year
Province
BAU:
    *Butane cost
    *PV cost
    *Grid emissions

2040 Phaseout: 
    *Butane cost
    *PV cost
    *Grid emissions


2030 Phaseout: 
    *Butane cost
    *PV cost
    *Grid emissions

    
2030 Phaseout PV: 
    *Butane cost
    *PV cost
    *Grid emissions

    
This ofcouse can be enhanced if we get the actual electricity generation cost from the grid


# How to iterate over scenarios